# Stability–Plasticity Report\n
\n
This notebook turns the app’s experiment log into portfolio-ready artifacts:\n
\n
- A Markdown table you can paste into `README.md`\n
- An interactive Plotly chart (optionally saved as HTML)\n
\n
**Input:** `data/experiments/stability_plasticity/results.csv`\n
\n
Run 3–5 feedback cycles in the Gradio app first (each incremental retrain appends one row).

In [ ]:
import os
import pandas as pd

RESULTS_CSV = os.environ.get("RFT_RESULTS_CSV_PATH", "data/experiments/stability_plasticity/results.csv")
RESULTS_CSV

In [ ]:
df = pd.read_csv(RESULTS_CSV)
df

## README Table\n
\n
This prints a compact table with the most relevant columns. Copy/paste into your README.

In [ ]:
view = df.copy()
for col in ("stability_accuracy", "plasticity_accuracy"):
    if col in view.columns:
        view[col] = pd.to_numeric(view[col], errors="coerce")

cols = [c for c in ["cycle", "feedback_samples", "stability_accuracy", "plasticity_accuracy", "learning_rate", "timestamp"] if c in view.columns]
table_df = view[cols].copy()
for col in ("stability_accuracy", "plasticity_accuracy"):
    if col in table_df.columns:
        table_df[col] = table_df[col].map(lambda x: "" if pd.isna(x) else f"{float(x):.3f}")

print(table_df.to_markdown(index=False))

## Plot (Stability vs Plasticity)\n
\n
This chart is useful for screenshots and quick interpretation.

In [ ]:
import plotly.express as px

plot_df = df.copy()
for col in ("stability_accuracy", "plasticity_accuracy"):
    if col in plot_df.columns:
        plot_df[col] = pd.to_numeric(plot_df[col], errors="coerce")

fig = px.line(
    plot_df,
    x="cycle",
    y=["stability_accuracy", "plasticity_accuracy"],
    markers=True,
    title="Stability vs Plasticity Across Feedback Cycles",
)
fig.update_layout(yaxis=dict(range=[0, 1]))
fig

## Save Plot as HTML (optional)\n
\n
If you want an artifact to link from the README, export an HTML plot.

In [ ]:
out_html = "data/experiments/stability_plasticity/plot.html"
os.makedirs(os.path.dirname(out_html), exist_ok=True)
fig.write_html(out_html, include_plotlyjs="cdn")
out_html